In [ ]:
%cd ..
import os
import time

In [ ]:
from apperception.database import database
from apperception.world import empty_world
from apperception.utils import F
from apperception.predicate import camera, objects
from optimized_ingestion.utils.preprocess import preprocess
database.connection
from optimized_ingestion.cache import disable_cache
# disable_cache()

In [ ]:
NUSCENES_PROCESSED_DATA = "NUSCENES_PROCESSED_DATA"
if NUSCENES_PROCESSED_DATA in os.environ:
    DATA_DIR = os.environ[NUSCENES_PROCESSED_DATA]
else:
    DATA_DIR = "/work/apperception/data/nuScenes/full-dataset-v1.0/Mini"
NUSCENES_RAW_DATA = "NUSCENES_RAW_DATA"
if NUSCENES_RAW_DATA in os.environ:
    RAW_DATA_DIR = os.environ[NUSCENES_RAW_DATA]
else:
    RAW_DATA_DIR = "/work/apperception/data/raw/nuScenes/full-dataset-v1.0/Mini"

In [ ]:
import cv2
from os.path import exists
def construct_video(frames, size=(1600, 900), base=False, vid_name=None, vid_prefix=False):
    unique_frames = []
    for f in frames:
        if f not in unique_frames:
            unique_frames.append(f)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    vid_name = "./outputs/fig13_"+vid_name if vid_name else f"./outputs/{'fig13_base' if base else 'fig13_optimized'}.mp4"
    display_video = cv2.VideoWriter(vid_name,fourcc, 1, size)
    for frame in unique_frames:
        if vid_prefix:
            img_path = os.path.join(RAW_DATA_DIR, 'sweeps/CAM_FRONT', frame)
            if not exists(img_path):
                img_path = os.path.join(RAW_DATA_DIR, 'samples/CAM_FRONT', frame)
        else:
            img_path = os.path.join(RAW_DATA_DIR, frame)
        img = cv2.imread(img_path)
        display_video.write(img)

    display_video.release()

In [ ]:
name = 'ScenicWorld'
world = empty_world(name=name)

obj1 = objects[0]
obj2 = objects[1]
cam = camera

world = world.filter(
    (obj1.id != obj2.id) &
    (F.like(obj1.type, 'car') | F.like(obj1.type, 'truck')) &
    (F.like(obj2.type, 'car') | F.like(obj2.type, 'truck')) &
    F.angle_between(F.facing_relative(cam.ego, F.road_direction(cam.ego)), -15, 15) &
    (F.distance(cam.ego, obj1.trans@cam.time) < 50) &
    (F.view_angle(obj1.trans@cam.time, cam.ego) < 70 / 2.0) &
    (F.distance(cam.ego, obj2.trans@cam.time) < 50) &
    (F.view_angle(obj2.trans@cam.time, cam.ego) < 70 / 2.0) &
    F.contains_all('intersection', [obj1.trans, obj2.trans]@cam.time) &
    F.angle_between(F.facing_relative(obj1.trans@cam.time, cam.ego), 40, 135) &
    F.angle_between(F.facing_relative(obj2.trans@cam.time, cam.ego), -135, -50) &
    (F.min_distance(cam.ego, 'intersection') < 10) &
    F.angle_between(F.facing_relative(obj1.trans@cam.time, obj2.trans@cam.time), 100, -100)
)


In [ ]:
preprocess(world, DATA_DIR, ['scene-0757-CAM_FRONT'], base=False)

In [ ]:
optimized_query_start = time.time()
id_time_camId_filename = world.get_id_time_camId_filename(2)
print(f'optimized query time: {time.time()-optimized_query_start}')

In [ ]:
optimized_results = [e[4] for e in id_time_camId_filename]
construct_video(optimized_results)

In [ ]:
preprocess(world, DATA_DIR)

In [ ]:
base_query_start = time.time()
id_time_camId_filename_base = world.get_id_time_camId_filename(2)
print(f'based query time: {time.time()-base_query_start}')

In [ ]:
id_time_camId_filename_base

In [ ]:
base_results = [e[4] for e in id_time_camId_filename_base]
construct_video(base_results, base=True)

In [ ]:
false_negative = [f for f in base_results if f not in optimized_results]
print(len(false_negative))
construct_video(false_negative, vid_name="false_negative.avi")

In [ ]:
false_positive = [f for f in optimized_results if f not in base_results]
print(len(false_positive))
construct_video(false_positive, vid_name="false_positive.avi")